### Travelling Salesman Problem

On a 4-node ring

In [3]:
import numpy as np

from scipy import sparse

coord = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
num_nodes = 4

def calc_distance():
    w = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            delta = coord[i] - coord[j]
            w[i, j] = (np.hypot(delta[0], delta[1]))
    w += w.T
    return w

w = calc_distance()

def Pauli(a,b):
    return a.astype(int)

def get_tsp_qubitops(penalty=1e5):
    num_qubits = num_nodes ** 2
    zero = np.zeros(num_qubits, dtype=np.bool)
    pauli_list = []
    shift = 0
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                continue
            for p in range(num_nodes):
                q = (p + 1) % num_nodes
                shift += w[i, j] / 4

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[i * num_nodes + p] = True
                pauli_list.append([-w[i, j] / 4, Pauli(zp, zero)])

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[j * num_nodes + q] = True
                pauli_list.append([-w[i, j] / 4, Pauli(zp, zero)])

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[i * num_nodes + p] = True
                zp[j * num_nodes + q] = True
                pauli_list.append([w[i, j] / 4, Pauli(zp, zero)])
                
    for i in range(num_nodes):
        for p in range(num_nodes):
            zp = np.zeros(num_qubits, dtype=np.bool)
            zp[i * num_nodes + p] = True
            pauli_list.append([penalty, Pauli(zp, zero)])
            shift += -penalty

    for p in range(num_nodes):
        for i in range(num_nodes):
            for j in range(i):
                shift += penalty / 2

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[i * num_nodes + p] = True
                pauli_list.append([-penalty / 2, Pauli(zp, zero)])

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[j * num_nodes + p] = True
                pauli_list.append([-penalty / 2, Pauli(zp, zero)])

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[i * num_nodes + p] = True
                zp[j * num_nodes + p] = True
                pauli_list.append([penalty / 2, Pauli(zp, zero)])

    for i in range(num_nodes):
        for p in range(num_nodes):
            for q in range(p):
                shift += penalty / 2

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[i * num_nodes + p] = True
                pauli_list.append([-penalty / 2, Pauli(zp, zero)])

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[i * num_nodes + q] = True
                pauli_list.append([-penalty / 2, Pauli(zp, zero)])

                zp = np.zeros(num_qubits, dtype=np.bool)
                zp[i * num_nodes + p] = True
                zp[i * num_nodes + q] = True
                pauli_list.append([penalty / 2, Pauli(zp, zero)])
    shift += 2 * penalty * num_nodes
    return pauli_list

def simplify_paulis(pl):
    """
    Merge the paulis (grouped_paulis) whose bases are identical but the pauli with zero coefficient would not be removed.
    """
    new_paulis = []
    new_paulis_table = {}
    for curr_paulis in pl:
        pauli_label = pz_str(curr_paulis[1])
        new_idx = new_paulis_table.get(pauli_label, None)
        if new_idx is not None:
            new_paulis[new_idx][0] += curr_paulis[0]
        else:
            new_paulis_table[pauli_label] = len(new_paulis)
            new_paulis.append(curr_paulis)
    return new_paulis

def pz_str(pz):
    """Output the Pauli label."""
    label = ''
    for z in pz:
        if not z:
            label = ''.join([label, 'I'])
        else:
            label = ''.join([label, 'Z'])
    return label

def paulis_to_matrix(pl):
    """
    Convert paulis to matrix, and save it in internal property directly.
    If all paulis are Z or I (identity), convert to dia_matrix.
    """
    p = pl[0]
    hamiltonian = p[0] * to_spmatrix(p[1])
    for idx in range(1, len(pl)):
        p = pl[idx]
        hamiltonian += p[0] * to_spmatrix(p[1])
    return hamiltonian

def to_spmatrix(p):
    """
    Convert Pauli to a sparse matrix representation (CSR format).
    Order is q_{n-1} .... q_0, i.e., $P_{n-1} \otimes ... P_0$
    Returns:
        scipy.sparse.csr_matrix: a sparse matrix with CSR format that
        represnets the pauli.
    """
    mat = sparse.coo_matrix(1)
    for z in p:
        if not z:  # I
            mat = sparse.bmat([[mat, None], [None, mat]], format='coo')
        else:  # Z
            mat = sparse.bmat([[mat, None], [None, -mat]], format='coo')
    return mat.tocsr() 

pl = get_tsp_qubitops()
pls = simplify_paulis(pl)
print(len(pl),len(pls))
m = paulis_to_matrix(pls)
#print(m)

for i in pls:
    print(pz_str(i[1]),i[0])


304 112
ZIIIIIIIIIIIIIII -200001.7071067812
IIIIIZIIIIIIIIII -200001.7071067812
ZIIIIZIIIIIIIIII 0.25
IZIIIIIIIIIIIIII -200001.7071067812
IIIIIIZIIIIIIIII -200001.7071067812
IZIIIIZIIIIIIIII 0.25
IIZIIIIIIIIIIIII -200001.7071067812
IIIIIIIZIIIIIIII -200001.7071067812
IIZIIIIZIIIIIIII 0.25
IIIZIIIIIIIIIIII -200001.7071067812
IIIIZIIIIIIIIIII -200001.7071067812
IIIZZIIIIIIIIIII 0.25
IIIIIIIIIZIIIIII -200001.7071067812
ZIIIIIIIIZIIIIII 0.25
IIIIIIIIIIZIIIII -200001.7071067812
IZIIIIIIIIZIIIII 0.25
IIIIIIIIIIIZIIII -200001.7071067812
IIZIIIIIIIIZIIII 0.25
IIIIIIIIZIIIIIII -200001.7071067812
IIIZIIIIZIIIIIII 0.25
IIIIIIIIIIIIIZII -200001.7071067812
ZIIIIIIIIIIIIZII 0.3535533905932738
IIIIIIIIIIIIIIZI -200001.7071067812
IZIIIIIIIIIIIIZI 0.3535533905932738
IIIIIIIIIIIIIIIZ -200001.7071067812
IIZIIIIIIIIIIIIZ 0.3535533905932738
IIIIIIIIIIIIZIII -200001.7071067812
IIIZIIIIIIIIZIII 0.3535533905932738
IZIIZIIIIIIIIIII 0.25
IIZIIZIIIIIIIIII 0.25
IIIZIIZIIIIIIIII 0.25
ZIIIIIIZIIIIIIII 0.25
IIIIZIII